In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!nvidia-smi


Tue Jun 10 21:45:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install -q transformers peft datasets accelerate bitsandbytes

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# 1. Install required packages (Run this in Kaggle terminal if needed)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 

2025-06-10 21:47:24.972647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749592045.190684      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749592045.261367      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# 2. Load conversational dataset
dataset = load_dataset("bavard/personachat_truecased")  # Example conversational dataset

# 3. Load pre-trained conversational model and tokenizer
# model_name = "mistralai/Mistral-7B-Instruct"  # Conversational model
# model_name = "meta-llama/Llama-2-7b-chat-hf"  # Alternative conversational model
model_name = "facebook/opt-1.3b"  # Smaller alternative conversational model


from huggingface_hub import login
login(token=login_token)


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)



README.md:   0%|          | 0.00/4.00k [00:00<?, ?B/s]

personachat_truecased.py:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/5.47k [00:00<?, ?B/s]

The repository for bavard/personachat_truecased contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bavard/personachat_truecased.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Repo card metadata block was not found. Setting CardData to empty.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [5]:
import datasets as ds
test_dataset = dataset['train'].select(range(10))  # Show the first example
test_dataset

Dataset({
    features: ['personality', 'candidates', 'history', 'conv_id', 'utterance_idx'],
    num_rows: 10
})

In [6]:
import random

random_number = random.randint(1000, 9999)


print(random_number)
def rand():
    return random.randint(1000, 9999)

rand()

2987


5444

In [7]:
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)



def tokenize_function(examples):
    # # Convert 'history' and 'candidates' into a single text string
    # history_text = " ".join(examples["history"]) 
    # candidates_text = " ".join(examples["candidates"]) 
    # personality_text = " ".join(examples["personality"]) 

    history_text = " ".join(examples.get("history", [])) 
    candidates_text = " ".join(examples.get("candidates", [])) 
    personality_text = " ".join(examples.get("personality", [])) 
    
    # Concatenate all parts to form a meaningful input
    text = f"Personality: {personality_text}\n History: {history_text}\n Candidates: {candidates_text}"
    tokens = len(tokenizer(text)["input_ids"])  
    # print("token_count", tokens, text, '\n')
    # return tokenizer(text, padding="max_length", truncation=True, max_length=2048)
    return tokenizer(
        text, 
        padding="max_length", 
        truncation=True, 
        max_length=2048,  # Reduce max length for efficiency
        return_tensors="pt"  # Return as PyTorch tensors for GPU acceleration
    )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device set to: ", device)
# tokenizer = tokenizer.to(device)
    

tokenized_datasets = dataset.map(tokenize_function, batched=False)


device set to:  cuda


Map:   0%|          | 0/131438 [00:00<?, ? examples/s]

Map:   0%|          | 0/7801 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['personality', 'candidates', 'history', 'conv_id', 'utterance_idx', 'input_ids', 'attention_mask'],
        num_rows: 131438
    })
    validation: Dataset({
        features: ['personality', 'candidates', 'history', 'conv_id', 'utterance_idx', 'input_ids', 'attention_mask'],
        num_rows: 7801
    })
})

In [9]:
# 4. Apply PEFT (LoRA) Configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, lora_config)
model.to(torch.device("cuda"))  # Move model to GPU

# # 5. Training Arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     run_name="llm_finetune_experiment",
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
#     num_train_epochs=3,
#     save_strategy="epoch",
#     logging_dir="./logs",
# )




PeftModel(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 2048, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
          (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTSdpaAttention(
                (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2048, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                   

In [10]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['personality', 'candidates', 'history', 'conv_id', 'utterance_idx', 'input_ids', 'attention_mask'],
        num_rows: 131438
    })
    validation: Dataset({
        features: ['personality', 'candidates', 'history', 'conv_id', 'utterance_idx', 'input_ids', 'attention_mask'],
        num_rows: 7801
    })
})

In [ ]:
# 5. Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="llm_finetune_experiment",  # Set a unique run name
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,  # Log every 10 steps
)

# 6. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# 7. Train the Model with More Granular Progress Bar
trainer.train()


# 8. Save the Fine-Tuned Model
# model.save_pretrained("/kaggle/working/my-finetuned-model")
# tokenizer.save_pretrained("/kaggle/working/my-finetuned-model")

print("Fine-tuning complete! Model saved to /kaggle/working/my-finetuned-model")
# 9. Evaluate the Model
eval_results = trainer.evaluate()

# Print evaluation metrics
print("Evaluation Metrics:")
for key, value in eval_results.items():
    print(f"{key}: {value}")


No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>